In [1]:
import psycopg2

from itertools import chain
import numpy as np
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import requests 
from datetime import datetime
import re
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
url=requests.get('''https://www.mtggoldfish.com/tournament_searches/create?utf8=%E2%9C%93&tournament_search%5Bname%5D=&tournament_search%5Bformat%5D=pioneer&tournament_search%5Bdate_range%5D=10%2F27%2F2019+-+01%2F22%2F2020&commit=Search''')

rawtext=BeautifulSoup(url.text,'html.parser')

In [3]:
rawtext.prettify()

'<!DOCTYPE html>\n<html lang="en" xmlns="http://www.w3.org/1999/xhtml">\n <head>\n  <title>\n   Advanced Tournament Search\n  </title>\n  <meta content="Search for Magic: the Gathering tournaments." name="description"/>\n  <meta content="search, new, tournament, tournaments, magic, mtg, magic the gathering, magic the gathering online, mtgo" name="keywords"/>\n  <meta content="Advanced Tournament Search" property="og:title"/>\n  <meta content="website" property="og:type"/>\n  <meta content="https://www.mtggoldfish.com/tournament_searches/create?utf8=%E2%9C%93&amp;tournament_search%5Bname%5D=&amp;tournament_search%5Bformat%5D=pioneer&amp;tournament_search%5Bdate_range%5D=10%2F27%2F2019+-+01%2F22%2F2020&amp;commit=Search" property="og:url"/>\n  <meta content="Search for Magic: the Gathering tournaments." property="og:description"/>\n  <meta content="summary" name="twitter:card"/>\n  <meta content="@mtggoldfish" name="twitter:site"/>\n  <link href="https://www.mtggoldfish.com/feed" rel="al

In [4]:
table=rawtext.find_all('tr')
[print(str(row) + "\n") for row in table]

<tr>
<th>Date</th>
<th>Name</th>
<th>Format</th>
</tr>

<tr>
<td>2020-01-21</td>
<td><a href="/tournament/30032">Pioneer Preliminary #12071860</a></td>
<td>pioneer</td>
</tr>

<tr>
<td>2020-01-20</td>
<td><a href="/tournament/30020">Pioneer League 2020-01-20</a></td>
<td>pioneer</td>
</tr>

<tr>
<td>2020-01-20</td>
<td><a href="/tournament/30026">Pioneer Preliminary #12071857</a></td>
<td>pioneer</td>
</tr>

<tr>
<td>2020-01-19</td>
<td><a href="/tournament/30016">Pioneer Challenge #12071844</a></td>
<td>pioneer</td>
</tr>

<tr>
<td>2020-01-18</td>
<td><a href="/tournament/30010">Pioneer Preliminary #12071837</a></td>
<td>pioneer</td>
</tr>

<tr>
<td>2020-01-17</td>
<td><a href="/tournament/30012">Pioneer Preliminary #12071834</a></td>
<td>pioneer</td>
</tr>

<tr>
<td>2020-01-16</td>
<td><a href="/tournament/29996">Pioneer League 2020-01-16</a></td>
<td>pioneer</td>
</tr>

<tr>
<td>2020-01-16</td>
<td><a href="/tournament/29999">Pioneer Preliminary #12071738</a></td>
<td>pioneer</td>
<

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [5]:
for entry in table[1:]:
    linktocards = entry.find('a').get('href')
    entrydate = entry.find('td').contents[0]
    

In [6]:
for entry in table[1:]:
       print(entry.find('td').contents[0])

2020-01-21
2020-01-20
2020-01-20
2020-01-19
2020-01-18
2020-01-17
2020-01-16
2020-01-16
2020-01-15
2020-01-14
2020-01-13
2020-01-13
2020-01-12
2020-01-12
2020-01-12
2020-01-12
2020-01-12
2020-01-12
2020-01-12
2020-01-12
2020-01-12
2020-01-11
2020-01-11
2020-01-10
2020-01-10
2020-01-09
2020-01-09
2020-01-08
2020-01-07
2020-01-06


In [7]:
table[1].find('a').get('href')

'/tournament/30032'

In [8]:
decklisturl=requests.get('https://www.mtggoldfish.com/'+str(table[1].find('a').get('href')))
decktext=BeautifulSoup(decklisturl.text,'html.parser')

In [9]:
decktext.prettify()

'<!DOCTYPE html>\n<html lang="en" xmlns="http://www.w3.org/1999/xhtml">\n <head>\n  <title>\n   Pioneer Preliminary #12071860 (Pioneer) Decks\n  </title>\n  <meta content="Pioneer Preliminary #12071860 (Pioneer) Magic: the Gathering deck lists with prices." name="description"/>\n  <meta content="pioneer preliminary #12071860, pioneer, results, deck, decks, deck list, tournament, daily event, finish, top 8, magic, mtg, magic the gathering, magic the gathering online, mtgo, daily events" name="keywords"/>\n  <meta content="Pioneer Preliminary #12071860 (Pioneer) Decks" property="og:title"/>\n  <meta content="website" property="og:type"/>\n  <meta content="https://www.mtggoldfish.com/tournament/pioneer-preliminary-12071860" property="og:url"/>\n  <meta content="Pioneer Preliminary #12071860 (Pioneer) Magic: the Gathering deck lists with prices." property="og:description"/>\n  <meta content="summary" name="twitter:card"/>\n  <meta content="@mtggoldfish" name="twitter:site"/>\n  <link href=

In [10]:
deckstuff = decktext.find('table',attrs={'class':'table table-condensed table-bordered table-tournament'})

In [11]:
decklinks = deckstuff.find_all('a')
deckurls=[]
for deck in decklinks:
    if str(deck.get('href')).lower().find('/deck/')!=-1:
        deckurls.append(deck.get('href'))

In [12]:
cardurl=requests.get('https://www.mtggoldfish.com'+str(deckurls[0]))
cardtext=BeautifulSoup(cardurl.text,'html.parser')

In [13]:
cardtext.prettify

<bound method Tag.prettify of <!DOCTYPE html>

<html lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>Izzet Ensoul by _Tia93_ Deck</title>
<meta content="Izzet Ensoul by _Tia93_ deck list with prices for Magic: the Gathering (MTG)." name="description"/>
<meta content="izzet ensoul, deck, decklist, wild slash, stonecoil serpent, ensoul artifact, pioneer preliminary #12071860, pioneer, price, magic, mtg, magic the gathering, magic the gathering online, mtgo, _tia93_" name="keywords"/>
<meta content="https://cdn1.mtggoldfish.com/images/gf/Wild%2BSlash%2B%255BFRF%255D.jpg" property="og:image"/>
<meta content="Izzet Ensoul  by _Tia93_ Deck" property="og:title"/>
<meta content="website" property="og:type"/>
<meta content="https://www.mtggoldfish.com/deck/2691027" property="og:url"/>
<meta content="Izzet Ensoul  by _Tia93_ deck list with prices for Magic: the Gathering (MTG)." property="og:description"/>
<meta content="summary" name="twitter:card"/>
<meta content="@mtggoldfish" n

In [49]:
cardquants=cardtext.find_all('td',attrs={'class':'deck-col-qty'})
cardquants=[int(quant.getText()) for quant in cardquants]

cardnamesintermed=[card for card in cardtext.find_all('td',attrs={'class':'deck-col-card'})]
cardnames = [card.find_all('a')[0].contents[0] for card in cardnamesintermed]
cardandquant=zip(cardnames,cardquants)

In [50]:
type(entrydate)

bs4.element.NavigableString

In [51]:
print(cardquants)

[4, 4, 4, 2, 2, 4, 2, 4, 4, 4, 4, 4, 1, 2, 4, 4, 4, 3, 2, 3, 2, 1, 2, 2, 1, 2, 4, 4, 4, 2, 2, 4, 2, 4, 4, 4, 4, 4, 1, 2, 4, 4, 4, 3, 2, 3, 2, 1, 2, 2, 1, 2, 4, 4, 4, 2, 2, 4, 2, 4, 4, 4, 4, 4, 1, 2, 4, 4, 4, 3, 2, 3, 2, 1, 2, 2, 1, 2]
